In [1]:
# setup vulkan
!mkdir -p /usr/share/vulkan/icd.d
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/nvidia_icd.json
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill/main/docker/10_nvidia.json
!mv nvidia_icd.json /usr/share/vulkan/icd.d
!mv 10_nvidia.json /usr/share/glvnd/egl_vendor.d/10_nvidia.json
!apt-get install -y --no-install-recommends libvulkan-dev
# dependencies
!pip install --upgrade mani_skill tyro

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libvulkan1
Recommended packages:
  mesa-vulkan-drivers | vulkan-icd
The following NEW packages will be installed:
  libvulkan-dev libvulkan1
0 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 1,020 kB of archives.
After this operation, 17.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan1 amd64 1.3.204.1-2 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan-dev amd64 1.3.204.1-2 [892 kB]
Fetched 1,020 kB in 1s (1,233 kB/s)
Selecting previously unselected package libvulkan1:amd64.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libvulkan1_1.3.204.1-2_amd64.deb ...
Unpacking libvulkan1:amd64 (1.3.204.1-2) ...
Selecting previously unselected package libvulkan-dev:amd64.
Preparing to un

In [5]:
!python -m mani_skill.utils.download_asset RoboCasa

<frozen runpy>:128: RuntimeWarning: 'mani_skill.utils.download_asset' found in sys.modules after import of package 'mani_skill.utils', but prior to execution of 'mani_skill.utils.download_asset'; this may result in unpredictable behaviour

-------------------------
/root/.maniskill/data does not exist. Create?
(y|n): y
3.48GiB [01:29, 41.8MiB/s]               
100% 76085/76085 [02:01<00:00, 625.54it/s]
Asset (RoboCasa) is successfully downloaded to /root/.maniskill/data/scene_datasets/robocasa_dataset.


In [4]:
!git clone https://github.com/SoshinK/darkstore_synthesizer.git

Cloning into 'darkstore_synthesizer'...
remote: Enumerating objects: 635, done.
remote: Total 635 (delta 0), reused 0 (delta 0), pack-reused 635 (from 1)
Receiving objects: 100% (635/635), 33.14 MiB | 19.61 MiB/s, done.
Resolving deltas: 100% (341/341), done.


In [11]:
from IPython.display import Video
Video("./videos_4_3_style0/0.mp4", embed=True, width=640)

In [9]:
from mani_skill.envs.tasks.empty_env import EmptyEnv
from mani_skill.envs.sapien_env import BaseEnv

from typing import Dict

from mani_skill.agents.robots.fetch import FETCH_WHEELS_COLLISION_BIT
from mani_skill.utils.building.ground import build_ground
from mani_skill.utils.registration import register_env
from mani_skill.utils import common, sapien_utils
import sapien
from mani_skill.sensors.camera import CameraConfig
import sys
import os
import json
import numpy as np
import gymnasium as gym
import torch
import mani_skill.envs
from tqdm import tqdm
from mani_skill.utils.wrappers import RecordEpisode
from transforms3d import quaternions
import random
import string
from robocasa_scene_builder import EmptyRoomFromRobocasa
import argparse

# parser = argparse.ArgumentParser(
#     description="Использование: python script.py <путь_к_JSON_файлу> <путь_к_assets> <style id (0-11)> [mapping_file]"
# )
# parser.add_argument("json_file_path", help="Путь к JSON файлу")
# parser.add_argument("assets_dir", help="Путь к assets")
# parser.add_argument("style_id", type=int, help="Style id (0-11)")
# parser.add_argument("mapping_file", nargs="?", default=None, help="Путь к mapping_file (опционально)")

# args = parser.parse_args()

# json_file_path = args.json_file_path
# assets_dir = args.assets_dir
# style_id = args.style_id
# mapping_file = args.mapping_file


json_file_path = "myscene_4_3.json"
assets_dir = "darkstore_synthesizer/models"
style_id = 0
mapping_file = None

def generate_random_string(length=10):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))


ENV_NAME = 'DarkstoreEnv'


CELL_SIZE = 1.55

def get_arena_data(x_cells=4, y_cells=5, height = 3):
    x_size = x_cells * CELL_SIZE
    y_size = y_cells * CELL_SIZE
    return {
        'meta': {
            'x_size': x_size,
            'y_size': y_size,
            'height': height
        },
        'arena_config': {
            'room': {
                'walls': [
                    {'name': 'wall', 'type': 'wall', 'size': [x_size / 2, height / 2, 0.02], 'pos': [x_size / 2, y_size, height / 2]},
                    {'name': 'wall_backing', 'type': 'wall', 'backing': True, 'backing_extended': [True, False], 'size': [x_size / 2, height / 2, 0.1], 'pos': [x_size / 2, y_size, height / 2]},

                    {'name': 'wall_front', 'type': 'wall', 'wall_side' : 'front', 'size': [x_size / 2, height / 2, 0.02], 'pos': [x_size / 2, 0, height / 2]},
                    {'name': 'wall_front_backing', 'type': 'wall', 'wall_side' : 'front', 'backing': True, 'size': [x_size / 2, height / 2, 0.1], 'pos': [x_size / 2, 0, height / 2]},

                    {'name': 'wall_left', 'type': 'wall', 'wall_side': 'left', 'size': [y_size / 2, height / 2, 0.02], 'pos': [0, y_size / 2, height / 2]},
                    {'name': 'wall_left_backing', 'type': 'wall', 'wall_side': 'left', 'backing': True, 'size': [y_size / 2, height / 2, 0.1], 'pos': [0, y_size / 2, height / 2]},

                    {'name': 'wall_right', 'type': 'wall', 'wall_side': 'right', 'size': [y_size / 2, height / 2, 0.02], 'pos': [x_size, y_size / 2, height / 2]},
                    {'name': 'wall_right_backing', 'type': 'wall', 'wall_side': 'right', 'backing': True, 'size': [y_size / 2, height / 2, 0.1], 'pos': [x_size, y_size / 2, height / 2]}
                ],
                'floor': [
                    {'name': 'floor', 'type': 'floor', 'size': [x_size / 2, y_size / 2, 0.02], 'pos': [x_size / 2, y_size / 2, 0.0]},
                    {'name': 'floor_backing', 'type': 'floor', 'backing': True, 'size': [x_size / 2, y_size / 2, 0.1], 'pos': [x_size / 2, y_size / 2, 0.0]}
                ]
            }
        }
    }

@register_env(ENV_NAME, max_episode_steps=200000)
class DarkstoreEnv(BaseEnv):
    SUPPORTED_REWARD_MODES = ["none"]
    """
    This is just a very smart environment for goida transformation from ss
    """
    IMPORTED_SS_SCENE_SHIFT = np.array([CELL_SIZE / 2, CELL_SIZE / 2, 0])

    def __init__(self, *args, robot_uids="panda", arena_config = None, meta = None, style_ids = 0, **kwargs):
        self.style_ids = style_ids
        self.arena_config = arena_config
        self.x_size = meta['x_size']
        self.y_size = meta['y_size']
        self.height = meta['height']
        super().__init__(*args, robot_uids=robot_uids, **kwargs)

    @property
    def _default_sensor_configs(self):
        pose = sapien_utils.look_at([1.25, -1.25, 1.5], [0.0, 0.0, 0.2])
        return [CameraConfig("base_camera", pose, 256, 256, np.pi / 2, 0.01, 100)]

    @property
    def _default_human_render_camera_configs(self):
        # pose = sapien_utils.look_at([0.2, 0.2, 4], [5, 5, 2])
        pose = sapien_utils.look_at([self.x_size - 0.1, self.y_size - 0.1, self.height], [0, 0, 0])
        return CameraConfig(
            "render_camera", pose=pose, width=512, height=512, fov=1, near=0.01, far=100
        )

    def _load_agent(self, options: dict):
        super()._load_agent(options, sapien.Pose(p=[-0.615, 0, 0]))

    def _load_scene(self, options: dict):
        self.scene_builder = EmptyRoomFromRobocasa(self, arena_config=self.arena_config)
        self.scene_builder.build(self.style_ids)
        self._load_scene_from_json(options)


    def _process_string(self, s):
        if '_' in s:
            return s.split('_',1)[0] + '.obj'
        if '.' in s:
            return s.split('.',1)[0] + '.obj'
        return s + '.obj'

    def _temp_process_string(self, s):
        for i, char in enumerate(s):
            if char in "_." or char.isdigit():
                return s[:i] + ".obj"
        return s + ".obj"

    def _get_absolute_matrix(self, node, nodes_dict):
        current_matrix = np.array(node[2]["matrix"])
        parent_name = node[0]
        while parent_name != "world":
            # print(f"Doing GOIDA IN PROCESS for name {node[1]} with parent {node[0]}")
            parent_node = nodes_dict[parent_name]
            parent_matrix = np.array(parent_node[2]["matrix"])
            current_matrix = parent_matrix @ current_matrix
            parent_name = parent_node[0]
        return current_matrix

    def _get_pq(self, matrix, origin):
        matrix = np.array(matrix)
        q = quaternions.mat2quat(matrix[:3,:3])
        p = matrix[:-1, 3] - origin
        return p, q

    def _load_scene_from_json(self, options: dict):
        super()._load_scene(options)
        self.actors = []

        scale = np.array(options.get("scale", [1.0, 1.0, 1.0]))
        origin = np.array(options.get("origin", [0.0, 1.0, 0.0]))

        with open(json_file_path, "r") as f:
            data = json.load(f)

        nodes_dict = {}
        for node in data["graph"]:
            nodes_dict[node[1]] = node

        asset_mapping = {}
        if mapping_file is not None:
            with open(mapping_file, "r") as f:
                asset_mapping = json.load(f)

        for node in data["graph"]:
            parent_name, obj_name, props = node
            if ('/' not in obj_name):
                abs_matrix = self._get_absolute_matrix(node, nodes_dict)

                p, q = self._get_pq(abs_matrix, origin)

                obj_name_to_check = self._temp_process_string(obj_name)[:-4]

                if obj_name_to_check in asset_mapping:
                    asset_file = os.path.join(assets_dir, asset_mapping[obj_name_to_check])
                else:
                    asset_file = ""


                if not os.path.exists(asset_file):
                    asset_file = os.path.join(assets_dir, self._temp_process_string(obj_name))

                if not os.path.exists(asset_file):
                    asset_file = os.path.splitext(asset_file)[0] + ".glb"

                if not os.path.exists(asset_file):
                    print("Not found file for " + obj_name + " =(" + " ( " + asset_file + " )")
                else:
                    # print("Found file for " + obj_name + " =)" + " ( " + asset_file + " )")
                    builder = self.scene.create_actor_builder()
                    builder.add_visual_from_file(filename=asset_file, scale=scale)
                    builder.set_initial_pose(sapien.Pose(p=p, q=q))



                    if obj_name.startswith('shelf'):
                        builder.add_nonconvex_collision_from_file(filename=asset_file, scale=scale)
                        actor = builder.build_static(name=obj_name)
                    else:
                        builder.add_convex_collision_from_file(filename=asset_file, scale=scale)
                        actor = builder.build(name=obj_name)

                    self.actors.append(actor)





    def _initialize_episode(self, env_idx: torch.Tensor, options: dict):
        if self.robot_uids == "fetch":
            qpos = np.array(
                [
                    0,
                    0,
                    0,
                    0.386,
                    0,
                    0,
                    0,
                    -np.pi / 4,
                    0,
                    np.pi / 4,
                    0,
                    np.pi / 3,
                    0,
                    0.015,
                    0.015,
                ]
            )
            self.agent.reset(qpos)
            self.agent.robot.set_pose(sapien.Pose([0.5, 0.5, 0.0]))

            # self.ground.set_collision_group_bit(
            #     group=2, bit_idx=FETCH_WHEELS_COLLISION_BIT, bit=1
            # )
        else:
            raise NotImplementedError


    def evaluate(self):
        return {}

    def _get_obs_extra(self, info: Dict):
        return dict()

with open(json_file_path, "r") as f: # big_scene , one_shelf_many_milk_scene , customize
    data = json.load(f)

n = data['meta']['n']
m = data['meta']['m']
arena_data = get_arena_data(x_cells=n, y_cells=m, height=4)

env = gym.make(ENV_NAME, robot_uids='fetch', style_ids = [style_id], num_envs=1, render_mode="rgb_array", enable_shadow=True, **arena_data)


env = RecordEpisode(
    env,
    f"./videos_{n}_{m}_style{style_id}", # the directory to save replay videos and trajectories to
    # on GPU sim we record intervals, not by single episodes as there are multiple envs
    # each 100 steps a new video is saved
    max_steps_per_video=100
)

# step through the environment with random actions
obs, _ = env.reset()


viewer = env.render()
if isinstance(viewer, sapien.utils.Viewer):
    viewer.paused = False
env.render()


for i in tqdm(range(100)):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(torch.zeros_like(torch.from_numpy(action)))

    env.render()
    # env.render_human() # will render with a window if possible
env.close()

2025-02-07 14:52:30,445 - mani_skill  - WARNING - Env DarkstoreEnv is already registered. Skip registration.
2025-02-07 14:52:31,282 - mani_skill  - WARNING - mani_skill is not installed with git.
100%|██████████| 100/100 [00:10<00:00,  9.15it/s]


In [ ]:
import numpy as np
import sapien

from mani_skill.envs.tasks import PickCubeEnv
from mani_skill.examples.motionplanning.panda.motionplanner import \
    PandaArmMotionPlanningSolver
from mani_skill.examples.motionplanning.panda.utils import (
    compute_grasp_info_by_obb, get_actor_obb)





def solve_by_coords(env: PickCubeEnv, target: Actor, goal_pose: sapien.Pose, vis=False):
    planner = PandaArmMotionPlanningSolver(
        env,
        debug=debug,
        vis=vis,
        base_pose=env.unwrapped.agent.robot.pose,
        visualize_target_grasp_pose=vis,
        print_env_info=False,
    )

    FINGER_LENGTH = 0.025
    env = env.unwrapped

    # retrieves the object oriented bounding box (trimesh box object)
    obb = get_actor_obb(target)

    approaching = np.array([0, 0, -1])
    # get transformation matrix of the tcp pose, is default batched and on torch
    target_closing = env.agent.tcp.pose.to_transformation_matrix()[0, :3, 1].cpu().numpy()
    # we can build a simple grasp pose using this information for Panda
    grasp_info = compute_grasp_info_by_obb(
        obb,
        approaching=approaching,
        target_closing=target_closing,
        depth=FINGER_LENGTH,
    )
    closing, center = grasp_info["closing"], grasp_info["center"]
    grasp_pose = env.agent.build_grasp_pose(approaching, closing, target.pose.sp.p)
    agent_z_reach_pose = env.agent.get_pose()
    agent_z_reach_pose[2] = grasp_pose[2]

    # -------------------------------------------------------------------------- #
    # Reach
    # -------------------------------------------------------------------------- #

    # reach_pose = grasp_pose * sapien.Pose([0, 0, -0.05])
    planner.move_to_pose_with_screw(agent_z_reach_pose)

    # -------------------------------------------------------------------------- #
    # Grasp
    # -------------------------------------------------------------------------- #
    planner.move_to_pose_with_screw(grasp_pose)

    # reach_pose = grasp_pose * sapien.Pose([0, 0, -0.05])
    planner.close_gripper()

    # -------------------------------------------------------------------------- #
    # Move to goal pose
    # -------------------------------------------------------------------------- #
    grasp_pose[:2] = goal_pose[:2]
    res = planner.move_to_pose_with_screw(grasp_pose)
    res = planner.move_to_pose_with_screw(goal_pose)

    planner.close()
    return res